# Web Scraping Tutorial

I've received interest from some people about web scraping, so I decided to make a tutorial to help you guys and gals get started!

Installation
1. XPath - Add On that helps you find absolute/relative paths of text on a webpage faster. I don't use Firefox so I am not sure how it works, but should work very similar.
    - Chrome https://chrome.google.com/webstore/detail/xpath-helper/hgimnogjllphhhkhlmebbmlgjoejdpjl
    - Firefox https://addons.mozilla.org/en-US/firefox/addon/xpath-checker/
2. conda install bs4 or pip install bs4
3. conda install lxml or pip install lxml
4. pip install scrapy

bs4 is BeautifulSoup4, which is a package that helps deal with HTML tags

lxml processes XML and HTML in Python

Scrapy allows us to use XPATH Selector

If you find yourself having problems scraping websites that have a ton of injected JavaScript or JQuery, you'll need to download a web driver:
    - Chrome: https://sites.google.com/a/chromium.org/chromedriver/downloads
    - Firefox: https://github.com/mozilla/geckodriver/releases
    - Safari: https://webkit.org/blog/6900/webdriver-support-in-safari-10/
    - IE: https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/

You'll also need to install Selenium
pip install selenium

Web drivers and Selenium allows you to access all the injected JavaScript and JQuery.

### *** FAIR WARNING ***
### Web scraping is not entirely legal. If you are accessing/scraping hundreds of pages within seconds, they will find out and you will get yourself banned! Please use with caution!

# Let's start scraping!

In [71]:
# First let's import everything we just installed!

import pandas as pd
import numpy as np
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
import requests
import time
from bs4 import BeautifulSoup

# If you need it, after downloading the webdriver, you'll need to access it
from selenium import webdriver
chromedriver = "/Users/edwardlee/Desktop/chromedriver"

In [72]:
# Let's scrape some Craigslist
# BE SURE THAT WHEN YOU DO .GET REQUEST, YOU AREN'T DOING IT CONSTANTLY, EVERY .GET RESPONSE TALKS TO THE WEBSITE.
# IF YOU DO IT MULTIPLE TIMES, THEY'LL CATCH ON!
# USE THE TIME COMMAND TO HAVE A BIT MORE RANDOMNESS TO IT, BUT ITS NOT 100% PROOF

response = requests.get('https://sfbay.craigslist.org/search/sfc/cta')
time.sleep(5*np.random.rand())
HTML = response.text
HTML[0:500]

u'\ufeff<!DOCTYPE html>\n\n<html class="no-js"><head>\n    <title>SF bay area cars &amp; trucks  - craigslist</title>\n\n    <meta name="description" content="SF bay area cars &amp; trucks  - craigslist">\n    <meta http-equiv="X-UA-Compatible" content="IE=Edge"/>\n    <link rel="canonical" href="https://sfbay.craigslist.org/search/sfc/cta">\n    <link rel="alternate" type="application/rss+xml" href="https://sfbay.craigslist.org/search/sfc/cta?format=rss" title="RSS feed for craigslist | SF bay area cars &amp;'

Above is the HTML for the cars and trucks in the sfbay. We want to find the titles and pricing! We will need to find the relative xpath for each tag. Using your xpath addon, highlight over the item you want and it should show you the absolute path. By using // you can narrow down the xpath to the item you want. An example below:

**Absolute Path**: ```/html[@class='js canvas draggable fileAPI geolocation hashChange matchMedia picture pushState placeholder no-touchCapable transitions localStorage']/body[@class='search has-map en desktop w1024 grid']/section[@id='page-top']/form[@id='searchform']/div[@id='sortable-results']/ul[@class='rows']/li[@class='result-row'][1]/p[@class='result-info']/span[@class='result-meta']/span[@class='result-price']```

**Relative Path**: ```//li[@class='result-row']//span[@class='result-price']```

The relative path gives me ALL the prices on the first page. We can save that into a variable.

In [73]:
prices = Selector(text=HTML).xpath("//li[@class='result-row']/p[@class='result-info']/span[@class='result-meta']/span[@class='result-price']/text()").extract()
prices[0:5]

[u'$3000', u'$2800', u'$9500', u'$4800', u'$11990']

In [76]:
# Now let's do the same thing with titles
titles = Selector(text=HTML).xpath("//li[@class='result-row']/p[@class='result-info']/a[@class='result-title hdrlnk']/text()").extract()
titles[0:5]

[u'2008 Chevy Tahoe LTZ 120k miles',
 u'2005 Kia Optima',
 u'2008 Mercedes c300 amg package 6 speed manual',
 u'2001 Volvo S80 LOW MILAGE 94K!!! VERY CLEAN!!!',
 u'2005 Ford F-250 Super Duty Diesel Lifted New Off Road Tires Tool Box!!']

In [77]:
print len(prices)
print len(titles)

92
100


However, we noticed that not every post has a price listed, but every title is listed! We will need to loop it 1 by 1 to make sure that we get everything

In [78]:
title_xpath = "//li[@class='result-row'][%s]/p[@class='result-info']/a[@class='result-title hdrlnk']/text()"
price_xpath = "//li[@class='result-row'][%s]/p[@class='result-info']/span[@class='result-meta']/span[@class='result-price']/text()"

# The [%s] sign is important! It'll allow us to fill in the value we need.

titles_all = []
price_all = []
for i in range(1,101):
    title_placeholder = title_xpath % i
    price_placeholder = price_xpath % i
    
    titles_all.append(Selector(text=HTML).xpath(title_placeholder).extract())
    price_all.append(Selector(text=HTML).xpath(price_placeholder).extract())
        
df = pd.DataFrame({'titles': titles_all, 'price': price_all})

In [79]:
# Completed!
df.head(10)

,price,titles
0,[$3000],[2008 Chevy Tahoe LTZ 120k miles]
1,[$2800],[2005 Kia Optima]
2,[$9500],[2008 Mercedes c300 amg package 6 speed manual]
3,[$4800],[2001 Volvo S80 LOW MILAGE 94K!!! VERY CLEAN!!!]
4,[$11990],[2005 Ford F-250 Super Duty Diesel Lifted New ...
5,[$995],[2006 Mitsubishi Eclipse 150k miles front dama...
6,[$14990],[2010 Acura TL Navigation Back Up Camera Leath...
7,[$4000],[2013 Kia Optima LX]
8,[$7495],[2006 *Volkswagen* *Jetta* Value Edition PZEV ...
9,[$23500],[For sale mercedes sl500]


BeautifulSoup is a great package to help you find information! Below is an example, I'll use the same HTML link.

In [80]:
soup = BeautifulSoup(HTML, "lxml")

In [81]:
soup.html.title

<title>SF bay area cars &amp; trucks  - craigslist</title>

In [82]:
soup.html.title.text

u'SF bay area cars & trucks  - craigslist'

In [83]:
element = soup.find_all('span', {'class': 'result-price'})

In [84]:
# You can now loop through this and put it into a list to put into a dataframe!
element[0].text

u'$3000'

BeautifulSoup4 is very powerful to help you clean up HTML and find specific things you need. I won't go over everything, but their documentation is really good: https://www.crummy.com/software/BeautifulSoup/bs4/doc/

Here's a quick tutorial on how to use webdriver. The code below is my code for scraping a dropdown box injected with JQuery. I could not scrape it regularly, so I used selenium's webdriver to get it.

In [85]:
browser = webdriver.Chrome(executable_path = chromedriver)

# Get all years for 1996-2016
browser.get('http://stats.nba.com/league/player/')
HTML = browser.page_source
year_data = Selector(text=HTML).xpath("//select[@name='Season']/option/text()").extract()

# Make sure to quit the webdriver browser or you'll end up having a bunch of opened browsers
browser.quit()

In [86]:
year_data

[u'2016-17',
 u'2015-16',
 u'2014-15',
 u'2013-14',
 u'2012-13',
 u'2011-12',
 u'2010-11',
 u'2009-10',
 u'2008-09',
 u'2007-08',
 u'2006-07',
 u'2005-06',
 u'2004-05',
 u'2003-04',
 u'2002-03',
 u'2001-02',
 u'2000-01',
 u'1999-00',
 u'1998-99',
 u'1997-98',
 u'1996-97']